In [1]:
import os
import json
import csv

import re

import  ftfy

from findJsonKeys import FindJsonKey 
from populateFlatByJson import populate 


# transform comment file

In [2]:
facebook_posts_dir="/mnt/BACKUP/dev/facebook-marianoaloi/your_activity_across_facebook/posts/"



findKeys=FindJsonKey()
def transform(msgPost:dict)-> dict:
    result={}
    result["title"]=ftfy.fix_text(msgPost["title"].replace("\n"," "))

    return result

with open(os.path.join(os.path.dirname(facebook_posts_dir),"posts.csv"), 'w', newline='', encoding='utf-8') as csvFile:
    csvSave = csv.DictWriter(csvFile, fieldnames="title,author,timestamp,comment,media,external_context".split(","))
    csvSave.writeheader()
    
    for root,folders,files in os.walk(facebook_posts_dir):
        folder=root.replace(facebook_posts_dir,"")
        for file in [file for file in files if "your_posts" in file]:
            with open(os.path.join(root,file), 'r') as user_file:
                parsed_json = json.load(user_file)
                # findKeys.makeDecision(parsed_json)
                parsed_json = [transform(msg) for msg in parsed_json if "data" in msg ]
        
                csvSave.writerows(transform(parsed_json))
                

print(findKeys.keys())                

_L_attachments
_L_attachments__L_data
_L_attachments__L_data__L_event
_L_attachments__L_data__L_event_end_timestamp
_L_attachments__L_data__L_event_name
_L_attachments__L_data__L_event_start_timestamp
_L_attachments__L_data__L_external_context
_L_attachments__L_data__L_external_context_name
_L_attachments__L_data__L_external_context_source
_L_attachments__L_data__L_external_context_url
_L_attachments__L_data__L_for_sale_item
_L_attachments__L_data__L_for_sale_item_category
_L_attachments__L_data__L_for_sale_item_created_timestamp
_L_attachments__L_data__L_for_sale_item_description
_L_attachments__L_data__L_for_sale_item_location
_L_attachments__L_data__L_for_sale_item_location_coordinate
_L_attachments__L_data__L_for_sale_item_location_coordinate_latitude
_L_attachments__L_data__L_for_sale_item_location_coordinate_longitude
_L_attachments__L_data__L_for_sale_item_location_name
_L_attachments__L_data__L_for_sale_item_marketplace
_L_attachments__L_data__L_for_sale_item_price
_L_attachmen